<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Output%20Parsers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Output parsers en Langchain**
---
Imagina que le preguntas a un LLM "¿Cuáles son los tres planetas más cercanos al sol?" y te responde: "Mercurio, Venus y la Tierra son los planetas más cercanos al sol". Si bien la respuesta es correcta para un humano, para que tu programa pueda usar esa información, lo ideal sería tenerla en un formato más manejable, como una lista o un objeto JSON. Aquí es donde entran en juego los Output Parsers.  

Los Output Parsers te permiten “forzar” o “guiar” al modelo para que devuelva la información según un formato deseado (por ejemplo, un JSON con campos específicos, una lista, etc).

LangChain ofrece una variedad de Output Parsers preconstruidos para diferentes necesidades. Aquí te presento algunos de los más comunes:

- **StrOutputParser:** Convierte la salida a string

-   **StructuredOutputParser:** Analiza la salida en una estructura predefinida con campos.
    
-   **OutputFixingParser:** Corrige errores en el parsing estructurado usando un LLM.
    
-   **CommaSeparatedListOutputParser:** Convierte la salida en una lista de strings separados por comas
    
-   **EnumOutputParser:** Espera que el LLM elija de un conjunto de opciones.
    
-   **BooleanOutputParser:** Analiza la salida como Verdadero o Falso.
    
-   **PydanticOutputParser:** Convierte la salida a un modelo de datos Pydantic.
    
-   **RegexParser:** Extrae información usando expresiones regulares.


OutputFixingParser: ?





In [2]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# # Importamos las clases necesarias para trabajar con cadenas
# from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.7 MB/s eta 0:00:00


## **2. StructuredOutputParser**

Permite analizar la salida del LLM en una estructura predefinida. Le proporcionas un esquema (generalmente una lista de ResponseSchema que definen los campos esperados) y el parser intenta extraer la información y mapearla a ese esquema.

**Casos de uso:** Extraer información específica de un texto, como atributos de un producto, detalles de un evento, o datos de contacto. Es muy versátil para obtener datos estructurados.

**Ventaja:** Flexible y permite definir la estructura esperada de la salida.

Veamos un ejemplo en el que el resultado será un diccionario estructurado con los campos nombre, edad y email, listo para ser utilizado en tu aplicación.

In [3]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

#Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
response_schemas = [
    ResponseSchema(name="nombre", description="El nombre del usuario"),
    ResponseSchema(name="edad", description="La edad del usuario"),
    ResponseSchema(name="email", description="El correo electrónico del usuario")
]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

**Obtener instrucciones de formato (opcional pero recomendado):** Muchos parsers tienen un método get\_format\_instructions() que devuelve texto que puedes incluir en tu prompt para guiar al LLM sobre el formato esperado.



In [4]:
# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"nombre": string  // El nombre del usuario\n\t"edad": string  // La edad del usuario\n\t"email": string  // El correo electrónico del usuario\n}\n```'

In [6]:
# Crear el prompt
prompt = PromptTemplate(
    template="Genera información de un usuario ficticio.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": format_instructions}
)

### Sobre las `partial variables`
Las **partial\_variables** dentro de un PromptTemplate son una forma de **pre-cargar o fijar ciertos valores dentro de la plantilla del prompt antes de que se proporcionen las variables de entrada principales, es decir cuando se crea el propmpt template y de antes de que se formatee**

Piénsalo de esta manera: un PromptTemplate es como una plantilla de texto con "huecos" que necesitas llenar para crear un prompt completo para el LLM. Hay dos formas principales de llenar estos huecos:

-   **input\_variables:** Estas son las variables que **cambian** cada vez que utilizas el prompt. Son los datos específicos que quieres que el LLM procese en cada llamada.
    
-   **partial\_variables:** Estas son las variables que tienen un valor **fijo** o **predefinido** para un uso particular del PromptTemplate. No cambian con cada llamada a la cadena o LLM que usa este prompt.
    

**En el codigo anterior:**



```
prompt = PromptTemplate(
            template="Genera información de un usuario ficticio.\n{format_instructions}\n",
            input_variables=[],
            partial_variables={"format_instructions": format_instructions}
```
En otras palabras, son una forma de incluir en el prompt template una parte fija, pero mediante una variable, lo que proporciona mas flexibilidad programatica.
En este caso metemos en el prompt, las instruciones que nos ha devuelto el parser.

In [16]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

respuesta = llm.invoke(prompt.format()) # la llamada no requiere ningun parámetro
respuesta_analizada = output_parser.parse(respuesta.content)
respuesta_analizada



{'nombre': 'Juan Pérez', 'edad': '28', 'email': 'juan.perez@example.com'}

## **3. ListOutputParser:**

Analiza la salida del LLM y la convierte en una lista de strings. Puedes especificar un separador para dividir el texto en elementos de la lista.
    
-   **Casos de uso:** Obtener listas de elementos, como nombres, ideas, pasos a seguir, o categorías.
    
-   **Ventaja:** Simple y efectivo para extraer listas.

Vamos a hacer un ejemplo en el que queremos obtener una lista de ingredientes para hacer una pizza casera.



In [20]:
from langchain.output_parsers import CommaSeparatedListOutputParser

# Crear el ListOutputParser
output_parser = CommaSeparatedListOutputParser()

# Crear el prompt
prompt = PromptTemplate(
    template="Genera una lista de ingredientes para hacer una pizza casera. Solo lista los ingredientes, uno por línea.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format()
respuesta = llm.invoke(input_prompt)

# Parsear la salida
respuesta_parseada = output_parser.parse(respuesta.content)

# Mostrar el resultado
print(respuesta_parseada)

['harina', 'agua', 'levadura', 'sal', 'aceite de oliva', 'tomate triturado', 'queso mozzarella', 'orégano', 'jamon', 'pepperoni', 'verduras (pimiento', 'cebolla', 'champiñones)']


## **4. EnumOutputParser**

Es útil cuando esperas que el LLM elija entre un conjunto predefinido de opciones (un enum de Python).

Casos de uso: Clasificación, selección de categorías, o elegir una opción de un menú.

Ventaja: Garantiza que la salida se encuentre dentro de un conjunto de valores válidos.Imaginemos que estamos construyendo un sistema de recomendación de libros, donde el modelo debe elegir un género literario de una lista predefinida y luego recomendar un libro de ese género.

❗ Este ejemplo no funciona ❓ -> porque este parser es mas bien para "forzar" a elegir al modelo una de las opciones. Obsrva que el "formato" que el parser espera (instructions) se incluye en el prompt para "forzar" la respuesta del modelo y luego el parser espera poder recoger la salida, pero a veces puede no estar bien

In [29]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

# Definir las opciones del enumerado
class ColoresOjos(Enum):
    MARRÓN = "marrón"
    AVELANA = "avellana"
    ÁMBAR = "ámbar"
    VERDE = "verde"
    AZUL = "azul"
    GRIS = "gris"


# Crear el EnumOutputParser
output_parser = EnumOutputParser(enum=ColoresOjos)

# Crear el prompt
prompt = PromptTemplate(
    template="De que color tenia o tiene los ojos esta persona: {persona}.",
    input_variables=["persona"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format(persona= "Frank Sinatra")
output = llm.invoke(input_prompt).content.lower()

parsed_output = output_parser.parse(output)

# Parsear la salida
print(f"El género elegido es: {parsed_output}")


OutputParserException: Response 'azul.' is not one of the expected values: ['marrón', 'avellana', 'ámbar', 'verde', 'azul', 'gris']
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [39]:
from langchain.output_parsers import EnumOutputParser


from enum import Enum

# Definir los géneros literarios como un Enum
class GenerosLiterarios(Enum):
    FANTASÍA = "fantasía"
    CIENCIA_FICCIÓN = "ciencia ficción"
    MISTERIO = "misterio"
    ROMANCE = "romance"
    HISTÓRICA = "novela histórica"
    NO_FICCIÓN = "no ficción"

# Crear el EnumOutputParser
output_parser = EnumOutputParser(enum=GenerosLiterarios)

# Crear el prompt
prompt = PromptTemplate(
    template="Elige un género literario de la siguiente lista: {generos_literarios}.\nLuego, recomienda un libro de ese género.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={
        "generos_literarios": ", ".join([g.value for g in GenerosLiterarios]),
        "format_instructions": output_parser.get_format_instructions()
    }
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format()
print(input_prompt)
respuesta = llm.invoke(input_prompt)

# Parsear la salida
try:
    genero_elegido = output_parser.parse(respuesta.content)
    print(f"El género elegido es: {genero_elegido.value}")

    # Ahora pedimos una recomendación de libro para el género elegido
    recomendacion_prompt = PromptTemplate(
        template="Recomienda un libro de género {genero}.",
        input_variables=["genero"]
    )
    recomendacion_input = recomendacion_prompt.format(genero=genero_elegido.value)
    recomendacion = llm.invoke(recomendacion_input)

    print(f"Recomendación: {recomendacion.content}")
except ValueError as e:
    print(f"Error: {e}")

Elige un género literario de la siguiente lista: fantasía, ciencia ficción, misterio, romance, novela histórica, no ficción.
Luego, recomienda un libro de ese género.
Select one of the following options: fantasía, ciencia ficción, misterio, romance, novela histórica, no ficción

Error: Response 'Elijo el género de **fantasía**. 

Te recomiendo el libro **"El nombre del viento"** de Patrick Rothfuss. Es la primera entrega de la trilogía "Crónica del asesino de reyes" y sigue la historia de Kvothe, un joven prodigio que se convierte en un legendario héroe. La narrativa es rica y envolvente, con un mundo bien construido lleno de magia, música y aventuras. ¡Espero que lo disfrutes!' is not one of the expected values: ['fantasía', 'ciencia ficción', 'misterio', 'romance', 'novela histórica', 'no ficción']
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [32]:
from langchain.output_parsers import RegexParser


# Definir la expresión regular para extraer nombre y edad
regex_pattern = r"Nombre: (?P<nombre>.+)\nEdad: (?P<edad>\d+)"

# Crear el RegexParser
output_parser = RegexParser(regex=regex_pattern, output_keys=["nombre", "edad"])

# Crear el prompt
prompt = PromptTemplate(
    template="Genera información de una persona ficticia, incluyendo su nombre y edad.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": ""}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format()
respuesta = llm.invoke(input_prompt)

# Parsear la salida
respuesta_parseada = output_parser.parse(respuesta.content)

# Mostrar el resultado
print(respuesta_parseada)

{'nombre': 'Valeria Mendoza  ', 'edad': '28'}
